In [1]:
from collections import OrderedDict
#コード上でもテキストファイル内(後から加える語根リスト.txt)でも後から加える語根のリストを編集できるようにした。(優先順位も決められる)
#コード上で後から加える語根のリストを編集(こちらがテキストファイル内(後から加える語根リスト.txt)で後から加える語根のリストよりも優先される)
add_vocab = OrderedDict([
    ("kulp", ["有罪", 40000]),#数字(デフォルトは語根の文字数*10000,変更可能)の大きい順に置換されていく。30001であれば3文字の語根(30000)よりも先に置換される。
    ("alkov", [30001]),#訳語がないものは語根のままで後から置換されないようにするため、unicode化された訳語がリストの頭に追加される。
    #すなわち、"alkov"は一旦'QQQ&#97;&#108;&#107;&#111;&#118;'に変換され、その後"alkov"に戻される。(この際、元の(変換前の)語根が小文字か大文字か頭だけ大文字かを確認し、変換後もそれに合わせるようにできている。)
    ("dosier", ["文件"]),##数字がないものは、語根の文字数*10000が自動的にリストに追加される。
    ("interlingvistik",[]),##数字も訳語もないものはunicode化された訳語と辞書のキーの長さ*10000が自動的にリストに追加される。
    ("kaj",[])])

#テキストファイル内(後から加える語根リスト.txt)で後から加える語根のリストを編集
with open('後から加える語根リスト(優先順位も決められる).txt', 'r') as file:
    for line in file:
        line = line.rstrip()
        data = line.split(',')
        if data[0] in add_vocab or data[0]=='':##"コード上で後から加える語根のリスト"を優先
            pass
        else:
            try:
                add_vocab[data[0]]=[data[1],int(data[2])]
            except:
                try:
                    add_vocab[data[0]]=[int(data[1])]
                except:
                    try:
                        add_vocab[data[0]]=[data[1]]
                    except:
                        add_vocab[data[0]]=[]
                
                
for key, value in add_vocab.items():
    if len(value) == 1:
        if isinstance(value[0],str):##リストに訳語が入っている場合はその後にキーである語根の文字数*10000の数字を追加する。
            char_count = len(key) * 10000
            add_vocab[key].append(char_count)
        elif  isinstance(value[0],(int, float)):##リストに数字が入っている場合は訳語を前に追加する。(訳語はunicode化)
            unicode_ka ='QQQ'+''.join(["&#"+str(ord(c))+";" for c in key])##"Q"にしたのはエスペラントには"Q"がなく、別の語根に食われて
            ## しまう心配がないから。一部でも食われると、unicode化した状態からもとに戻れなくなる。　また、変換時は、qqq,QQQ,Qqqに変えることによって小文字か大文字か頭だけ大文字かの情報を保存している。
            add_vocab[key].insert(0,unicode_ka)

    elif  len(value) == 0:##リストに何も入っていない(空)場合は、訳語も数字も追加する。
        unicode_ka ='QQQ'+''.join(["&#"+str(ord(c))+";" for c in key])
        char_count = len(key) * 10000

        add_vocab[key].append(unicode_ka)
        add_vocab[key].append(char_count)
        
print(add_vocab)##うまく行っているか確認


OrderedDict([('kulp', ['有罪', 40000]), ('alkov', ['QQQ&#97;&#108;&#107;&#111;&#118;', 30001]), ('dosier', ['文件', 60000]), ('interlingvistik', ['QQQ&#105;&#110;&#116;&#101;&#114;&#108;&#105;&#110;&#103;&#118;&#105;&#115;&#116;&#105;&#107;', 150000]), ('kaj', ['QQQ&#107;&#97;&#106;', 30000]), ('cxar', ['なので', 40001]), ('brasil', ['QQQ&#98;&#114;&#97;&#115;&#105;&#108;', 60001]), ('rektangul', ['长方形', 90001]), ('malm', ['malm', 20001]), ('avis', ['abes', 10001]), ('ari', ['ari', 20001]), ('din', ['din', 20000]), ('devis', ['必is', 40001])])


In [2]:
# Esperantoの語根と意味をロードする
roots_dict = OrderedDict([])

with open('全語根と一部の語根に対応する漢字のリスト.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if ',' in line:
            root, meaning = line.split(',', 1)
            roots_dict[root] = [meaning,len(root)*10000]##訳語と数字(語根の文字数*10000)を追加
        else:
            # 対応する漢字が存在しない場合の処理
            unicode_str = 'QQQ' + ''.join(["&#"+str(ord(c))+";" for c in line])#対応する漢字が存在しない場合はそのままにして、あとから別の語根によって漢字化されてしまわないようにする。
            roots_dict[line] = [unicode_str,len(line)*10000]#すなわち、一旦アルファベットをunicode化したものを対応させ、あとから元のアルファベットに戻すようにする。
            
for key, value in add_vocab.items():##後から加える語根のリストを追加 無論重複している語根は更新される。
    roots_dict[key]=value
    
# 数字(デフォルトは語根の文字数*10000,変更可能)の順にソート
sorted_items = sorted(roots_dict.items(), key=lambda x: -x[1][1])###数字の大きい順に並べる。数字の大きい順に語根を置換していく。
sorted_vocab = OrderedDict(sorted_items)##ソートでリスト型になったのを再度OrderedDict型に戻す。
# print(sorted_vocab)


In [3]:
##振り仮名(アルファベット)をつけるための処理 html形式でルビをふる。 
import copy
sorted_keys = list(sorted_vocab.keys())
sorted_vocab_copy=copy.deepcopy(sorted_vocab)#コピーを作っておく。 deepcopyでないと浅いコピー(同期)になってしまう。unicode化したものを戻すときに用いる。
for key in sorted_keys:
    if "&#" not in sorted_vocab[key][0]:##["mal","<qqq&#114;&#117;&#98;y>反<rt>qqq&#109;&#97;&#108;</rt>"]
        key_2="QQQ"+''.join(["&#"+str(ord(c))+";" for c in key])#振り仮名が後で変換されてしまうことを防ぐため。
        sorted_vocab[key][0]='<ruby>{}<rt>{}</rt></ruby>'.format(sorted_vocab[key][0],key_2)
        
for key in sorted_keys:##["mal","qqq&#109;&#97;&#108;"]
    if "&#" not in sorted_vocab_copy[key][0]:
        key_2="QQQ"+''.join(["&#"+str(ord(c))+";" for c in key])#振り仮名が後で変換されてしまうことを防ぐため。
        sorted_vocab_copy[key][0]=key_2 

In [4]:
import json
# OrderedDictをJSONファイルに保存
with open('sorted_vocab.json', 'w') as f:
    json.dump(sorted_vocab, f, indent=4)
with open('sorted_vocab_copy.json', 'w') as f:
    json.dump(sorted_vocab_copy, f, indent=4)


In [5]:
##もどす
with open('sorted_vocab.json', 'r') as f:
    sorted_vocab = json.load(f, object_pairs_hook=OrderedDict)
with open('sorted_vocab_copy.json', 'r') as f:
    sorted_vocab_copy = json.load(f, object_pairs_hook=OrderedDict)
    
sorted_keys = list(sorted_vocab.keys())   

In [6]:
import re

def replacer(s, new_word):###置換する語根すべてについて小文字か大文字か頭だけ大文字かを確認し、変換後もそれに合わせるための関数。 
    if s.islower():##それぞれqqq,QQQ,Qqqから始まるunicode化した状態に変化させることで、元の語根の大文字小文字の情報を保存している。　あくまで近似である。 (頭だけ大文字の場合:Qqq&#110;&#117;&#114;)
        return new_word.lower()
    elif s.isupper():
        return new_word.upper()
    elif s[0].isupper():
        return new_word.capitalize()
    else:
        return new_word

def multi_smart_replace(text, replacements):##replacerを使って大文字小文字の状態を保ったままの変換
    # for old_word, new_word in sorted_replacements:
    for old_word, new_word in replacements.items():
        text = re.sub(re.escape(old_word), lambda match: replacer(match.group(0), new_word), text, flags=re.IGNORECASE)

    # print(sorted_replacements)
    return text





def multiple_replace(text, replace_dict):
    # 各キーを正規表現のパターンとして結合する
    pattern = re.compile("|".join([re.escape(k) for k in replace_dict.keys()]), re.M)

    # 置換関数
    def sub_func(match):
        return replace_dict[match.group(0)]

    # 置換を実行
    return pattern.sub(sub_func, text)




In [10]:
# with open("sorted_vocab_copy.txt", "w", encoding='utf-8') as file:
#     for key, value in sorted_vocab_copy.items():
#         file.write(f"{key}: {value}\n")


##sorted_vocab,sorted_vocab_copy,sorted_keysをファイルとして予め用意しておくと良い。

In [11]:
esperanto_to_x = {"ĉ": "cx","ĝ": "gx","ĥ": "hx","ĵ": "jx","ŝ": "sx","ŭ": "ux","Ĉ": "Cx","Ĝ": "Gx","Ĥ": "Hx","Ĵ": "Jx","Ŝ": "Sx","Ŭ": "Ux",
"c^": "cx","g^": "gx","h^": "hx","j^": "jx","s^": "sx","u^": "ux","C^": "Cx","G^": "Gx","H^": "Hx","J^": "Jx","S^": "Sx","U^": "Ux"}
x_to_jijofu={'cx': 'ĉ', 'gx': 'ĝ', 'hx': 'ĥ', 'jx': 'ĵ', 'sx': 'ŝ', 'ux': 'ŭ', 'Cx': 'Ĉ',
             'Gx': 'Ĝ', 'Hx': 'Ĥ', 'Jx': 'Ĵ', 'Sx': 'Ŝ', 'Ux': 'Ŭ'}
x_to_hat={'cx': 'c^', 'gx': 'g^', 'hx': 'h^', 'jx': 'j^', 'sx': 's^', 'ux': 'u^', 'Cx': 'C^',
          'Gx': 'G^', 'Hx': 'H^', 'Jx': 'J^', 'Sx': 'S^', 'Ux': 'U^'}

In [12]:
# process the Esperanto text

## 原文がX型か字上符型か^型かを確認(0はX型、1は字上符型、2は^型)
letter_type=1

with open('例文(input).txt', 'r', encoding='utf-8') as f:
    text = f.read()

text1 = multiple_replace(text, esperanto_to_x)

replacements_1 = {key: sorted_vocab[key][0] for key in sorted_keys}
text2 = multi_smart_replace(text1, replacements_1)

# Build the unicode replacements dictionary
replacements_2 = {sorted_vocab_copy[key][0]: key for key in sorted_keys if '&#' in sorted_vocab_copy[key][0]}
text3 = multi_smart_replace(text2, replacements_2)


if letter_type==1:
    text4 = multiple_replace(text3, x_to_jijofu)
elif letter_type==2:
    text4 = multiple_replace(text3, x_to_hat)

with open('語根に対応する漢字によって置換された文章(output).html', 'w', encoding='utf-8') as f:
    f.write(text4)

In [18]:
##倍速並列処理
import multiprocessing
# process the Esperanto text

## 原文がX型か字上符型か^型かを確認(0はX型、1は字上符型、2は^型)
letter_type=1


def process_segment(lines, letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2):
    # 文字列のリストを結合してから置換処理を実行
    segment = '\n'.join(lines)
    segment = multiple_replace(segment, esperanto_to_x)
    segment = multi_smart_replace(segment, replacements_1)
    segment = multi_smart_replace(segment, replacements_2)

    if letter_type == 1:
        segment = multiple_replace(segment, x_to_jijofu)
    elif letter_type == 2:
        segment = multiple_replace(segment, x_to_hat)

    return segment

def parallel_process(text, num_processes, letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2):
    # テキストを行で分割
    lines = text.split('\n')
    num_lines = len(lines)
    lines_per_process = num_lines // num_processes

    # 各プロセスに割り当てる行のリストを決定
    ranges = [(i * lines_per_process, (i + 1) * lines_per_process) for i in range(num_processes)]
    ranges[-1] = (ranges[-1][0], num_lines)  # 最後のプロセスが残り全てを処理

    with multiprocessing.Pool(processes=num_processes) as pool:
        # 並列処理を実行
        results = pool.starmap(process_segment, [(lines[start:end], letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2) for start, end in ranges])

    # 結果を結合
    return '\n'.join(result for result in results)

# テキストの読み込み
with open('例文(input).txt', 'r', encoding='utf-8') as f:
    text = f.read()###文字数が増えれば増えるほど、処理時間がプロセス数(CPUの個数)に反比例するようになる。

# 置換辞書を準備
replacements_1 = {key: sorted_vocab[key][0] for key in sorted_keys}
replacements_2 = {sorted_vocab_copy[key][0]: key for key in sorted_keys if '&#' in sorted_vocab_copy[key][0]}

# 並列処理の実行
processed_text = parallel_process(text, 8, letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2)

# 結果をファイルに書き込み
with open('語根に対応する漢字によって置換された文章(output).html', 'w', encoding='utf-8') as f:
    f.write(processed_text)


In [83]:
##倍速並列処理 メモリの何かを変えたようだが、大した変化なし プロセス数が大きくなったら効果あるかも
import multiprocessing
from itertools import islice

def process_segment(start, end, letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2, shared_text):
    # 指定された範囲のテキストを取得
    segment = '\n'.join(islice(shared_text, start, end))
    segment = multiple_replace(segment, esperanto_to_x)
    segment = multi_smart_replace(segment, replacements_1)
    segment = multi_smart_replace(segment, replacements_2)

    if letter_type == 1:
        segment = multiple_replace(segment, x_to_jijofu)
    elif letter_type == 2:
        segment = multiple_replace(segment, x_to_hat)

    return segment

def parallel_process(text, num_processes, letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2):
    # テキストを行で分割
    lines = text.split('\n')
    num_lines = len(lines)

    # マネージャを使用して共有メモリ上にテキストを配置
    with multiprocessing.Manager() as manager:
        shared_text = manager.list(lines)
        tasks = []

        for i in range(num_processes):
            start = i * num_lines // num_processes
            end = (i + 1) * num_lines // num_processes
            tasks.append((start, end, letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2, shared_text))

        with multiprocessing.Pool(processes=num_processes) as pool:
            results = pool.starmap(process_segment, tasks)

    return '\n'.join(result for result in results)
# テキストの読み込み
with open('例文(input).txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 置換辞書を準備
replacements_1 = {key: sorted_vocab[key][0] for key in sorted_keys}
replacements_2 = {sorted_vocab_copy[key][0]: key for key in sorted_keys if '&#' in sorted_vocab_copy[key][0]}

# 並列処理の実行
processed_text = parallel_process(text, 4, letter_type, esperanto_to_x, x_to_jijofu, x_to_hat, replacements_1, replacements_2)

# 結果をファイルに書き込み
with open('語根に対応する漢字によって置換された文章(output).html', 'w', encoding='utf-8') as f:
    f.write(processed_text)
